In [110]:
from dotenv import load_dotenv
from langgraph.graph import StateGraph,END,add_messages,START
from langgraph.types import Command,interrupt
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from typing import TypedDict,Annotated
from langchain_community.vectorstores import Chroma
load_dotenv()
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)


In [115]:
from langchain.schema import Document

docs = [
    Document(
        page_content=""" 
If I had to pick one, it would be One Piece. The series stands out for its insanely deep lore, interconnected stories, and a massive, vibrant world that continues to expand. The character development is top-notch—Luffy and his crew grow meaningfully over time, and even side characters have rich, detailed backstories. Despite having over 1000 episodes, the plot consistently raises the stakes and delivers satisfying payoffs, which is rare for such long-running series. One Piece also explores themes like freedom, dreams, loyalty, and moral complexity in a nuanced way. That said, Naruto is an incredible series in its own right, especially known for its emotional arcs, legendary battles, and strong coming-of-age narrative. If you prefer tighter pacing and more grounded emotional storytelling—particularly in the original Naruto—then that might be the better fit for you.
        """
        ,metadata={"source":"mygpt/chatgpt"} #test
    ),
        Document(
        page_content=""" 
            naruto is a ninja legend
        """
        ,metadata={"source":"mygpt/naruto"} #test
    ),
       Document(
        page_content=""" 
            luffy is the best main character
        """
        ,metadata={"source":"mygpt/luffy"} #test
    ),
    
    Document(
        page_content="""
Vector databases are optimized systems designed for storing and retrieving high-dimensional vector representations of data.
These vectors are often the result of embedding text, images, or other data modalities using machine learning models.
Instead of matching exact keywords, vector databases enable semantic search by comparing the similarity between vectors.
This approach is especially useful in natural language processing, recommendation engines, and other AI applications where contextual meaning matters.
""",
        metadata={"source": "wikipedia/vector_databases"}
    ),
    Document(
        page_content="""
FAISS, which stands for Facebook AI Similarity Search, is an open-source library developed by Facebook AI Research.
It provides efficient algorithms for clustering and nearest neighbor search on dense vectors.
FAISS supports both CPU and GPU implementations, allowing it to scale to billions of vectors.
It is widely used in industry for powering semantic search, recommendation systems, and AI-powered assistants.
Its support for indexing structures like IVF, HNSW, and PQ makes it versatile for trade-offs between speed and accuracy.
""",
        metadata={"source": "docs/faiss"}
    ),
    Document(
        page_content="""
Chroma is a modern, open-source vector database built specifically for large language model applications.
It provides a simple API for adding documents, querying them, and managing embeddings.
Chroma integrates seamlessly with LangChain and supports persistent storage, making it ideal for long-term document management.
It includes features like namespace separation, filtering by metadata, and fast similarity search across millions of embeddings.
Developers can choose to embed documents using OpenAI, HuggingFace, or custom models.
""",
        metadata={"source": "chroma/overview"}
    ),
    Document(
        page_content="""
In modern AI systems, raw data like text, images, and audio are transformed into embeddings—dense numerical representations of the data.
These embeddings capture semantic meaning and allow for efficient similarity comparison.
To generate embeddings, models like OpenAI's `text-embedding-ada-002` or HuggingFace's `sentence-transformers` are commonly used.
Once generated, these vectors are stored in a vector database for real-time semantic retrieval.
This technique powers many LLM use cases such as RAG (Retrieval-Augmented Generation) and intelligent assistants.
""",
        metadata={"source": "course/embeddings101"}
    ),
    Document(
        page_content="""
Similarity search is the core operation in vector databases, where the goal is to find vectors in the database that are most similar to a query vector.
This is typically done using metrics like cosine similarity, Euclidean distance, or dot product.
The result is a ranked list of documents that are most relevant to the query in terms of semantic meaning.
To improve performance at scale, vector databases use indexing techniques like HNSW (Hierarchical Navigable Small World graphs).
These indexes reduce the number of distance computations and allow real-time response for millions of documents.
""",
        metadata={"source": "notes/similarity_search"}
    ),
]

db = Chroma.from_documents(documents=docs,embedding=embeddings)

In [116]:
embeddings.embed_documents("hello there")

[[-0.05710163339972496,
  0.09410133212804794,
  -0.042961277067661285,
  0.015190447680652142,
  0.003590963315218687,
  0.019870677962899208,
  0.08121766149997711,
  0.037185460329055786,
  -0.010115430690348148,
  -0.0536881648004055,
  0.061433710157871246,
  -0.09885638952255249,
  -0.022344831377267838,
  -0.06459888815879822,
  -0.012610557489097118,
  -0.002336295321583748,
  -0.07551310211420059,
  -0.00948034506291151,
  -0.059548523277044296,
  -0.07525581866502762,
  -0.06103133037686348,
  0.023791320621967316,
  0.022302623838186264,
  0.017092255875468254,
  -0.05245206505060196,
  0.05123639106750488,
  0.008438472636044025,
  0.06214277073740959,
  -0.01946290209889412,
  -0.09673580527305603,
  -0.0029820140916854143,
  0.002574898535385728,
  0.1014222651720047,
  -0.01917099952697754,
  0.0007710480131208897,
  -0.025375522673130035,
  -0.023148175328969955,
  -0.10248351842164993,
  -0.014852466061711311,
  0.018474861979484558,
  -0.03515508770942688,
  -0.039103

In [117]:
retriever = db.as_retriever(search_type="mmr",search_kwargs={"k":3})
retriever.invoke("Vector databases are ")

[Document(metadata={'source': 'wikipedia/vector_databases'}, page_content='\nVector databases are optimized systems designed for storing and retrieving high-dimensional vector representations of data.\nThese vectors are often the result of embedding text, images, or other data modalities using machine learning models.\nInstead of matching exact keywords, vector databases enable semantic search by comparing the similarity between vectors.\nThis approach is especially useful in natural language processing, recommendation engines, and other AI applications where contextual meaning matters.\n'),
 Document(metadata={'source': 'chroma/overview'}, page_content='\nChroma is a modern, open-source vector database built specifically for large language model applications.\nIt provides a simple API for adding documents, querying them, and managing embeddings.\nChroma integrates seamlessly with LangChain and supports persistent storage, making it ideal for long-term document management.\nIt includes

In [118]:
from langchain_core.prompts import ChatPromptTemplate

prompt_temp = ChatPromptTemplate.from_template(
    """
        Answer the following question using this context:{context}
        question:{question}
     """
)

In [119]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = prompt_temp | llm

In [120]:
from typing import TypedDict,Annotated
from langchain_core.messages import AIMessage,HumanMessage,BaseMessage
from langgraph.graph import add_messages,StateGraph,END

class AgentState(TypedDict):
    messages: list[BaseMessage]
    documents: list[Document]
    on_topic : str

In [154]:
from pydantic import BaseModel,Field

class QuestionInTopics(BaseModel):
    """Boolean value to check if the given question related to the topics"""

    val: str=Field(description="Question is about topics listed? if yes -> yes else -> no")


In [155]:
from langchain_groq import ChatGroq

def question_classifier_node(state: AgentState):
    topics_prompt = """
    Check if the given question is related to any of the topics
    # === Topics ===
    # 1. Vector Databases
    # 2. FAISS Library
    # 3. ChromaDB Overview
    # 4. Embeddings & Models
    # 5. Similarity Search
    # 6. Retrieval-Augmented Generation (RAG)
    # 7. LangChain Overview
    # 8. HuggingFace Transformers
    # 9. HNSW Indexing
    # 10.OpenAI Embedding API
    # 11.Naruto
    # 12.Luffy
    

    if given question is related even if slightly related to the above topics give yes else give no(respond with only yes or no)
    """
    topics_template = ChatPromptTemplate.from_messages([
        ("system",topics_prompt),
        ("human","quesion: {question}")
    ])
    llm = ChatGroq(model="llama-3.1-8b-instant")
    llm = llm.with_structured_output(QuestionInTopics)
    llm_chain = {
        "question":lambda x:x["question"]
        } | topics_template | llm 
    output=llm_chain.invoke({"question":state["messages"][-1]})
    state["on_topic"] = output.val
    print(state["on_topic"])
    return state

In [156]:
def on_topic_route(state:AgentState):
    return "retrieve_node" if state["on_topic"] == "yes" else "off_topic_response"


def retrieve_node(state:AgentState):
    docs = retriever.invoke(state["messages"][-1].content)
    state["documents"]=docs
    return state

def off_topic_response(state:AgentState):
    return state["messages"].append(AIMessage(content="Sorry the given question is off my boundaries i cannot answer."))

def answer_node(state:AgentState):
    formatted_docs = format_docs(state["documents"])

    answer = rag_chain.invoke({"context":formatted_docs,"question":state["messages"]})

    return state["messages"].append(answer)


In [157]:
graph = StateGraph(AgentState)

graph.add_node("question_classifier_node",question_classifier_node)
graph.add_node("retrieve_node",retrieve_node)
graph.add_node("answer_node",answer_node)
graph.add_node("off_topic_response",off_topic_response)

graph.add_edge(START,"question_classifier_node")
graph.add_conditional_edges("question_classifier_node",on_topic_route,{"retrieve_node":"retrieve_node","off_topic_response":"off_topic_response"})
graph.add_edge("retrieve_node","answer_node")

app = graph.compile()

In [158]:
initial_state = {
    "messages":[HumanMessage(content="why luffy is better than naruto")],
    "documents":[],
    "on_topic":""
}
print(app.invoke(initial_state))

yes
{'messages': [HumanMessage(content='why luffy is better than naruto', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on the provided context, it states that "luffy is the best main character." The context does not provide reasons why Luffy is better than Naruto, only that Naruto is "a ninja legend."', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--9d431a5d-482c-499a-b527-9e0c6ca9ef8d-0', usage_metadata={'input_tokens': 178, 'output_tokens': 40, 'total_tokens': 381, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 163}})], 'documents': [Document(metadata={'source': 'mygpt/luffy'}, page_content=' \n            luffy is the best main character\n        '), Document(metadata={'source': 'mygpt/naruto'}, page_content=' \n            naruto is a ninja legend\n        '), Document(metadata={'

In [159]:
from langchain_core.documents import Document

doc = Document(page_content="This is the document text.", metadata={"source": "myfile.pdf"})

# Get the raw text
text = doc.page_content
print(text)  # Output: This is the document text.


This is the document text.


In [161]:
retriever.invoke("why one piece is better than naruto")

[Document(metadata={'source': 'mygpt/chatgpt'}, page_content=' \nIf I had to pick one, it would be One Piece. The series stands out for its insanely deep lore, interconnected stories, and a massive, vibrant world that continues to expand. The character development is top-notch—Luffy and his crew grow meaningfully over time, and even side characters have rich, detailed backstories. Despite having over 1000 episodes, the plot consistently raises the stakes and delivers satisfying payoffs, which is rare for such long-running series. One Piece also explores themes like freedom, dreams, loyalty, and moral complexity in a nuanced way. That said, Naruto is an incredible series in its own right, especially known for its emotional arcs, legendary battles, and strong coming-of-age narrative. If you prefer tighter pacing and more grounded emotional storytelling—particularly in the original Naruto—then that might be the better fit for you.\n        '),
 Document(metadata={'source': 'mygpt/naruto'}